In [103]:
# Loading Visualization

output the estimation



//output the estimation
		out_file_ << ukf.x_(0) << "\t";	//pos1 - est
		out_file_ << ukf.x_(1) << "\t";	//pos2 - est
		out_file_ << ukf.x_(2) << "\t";	//vel_abs -est
		out_file_ << ukf.x_(3) << "\t";	//yaw_angle -est
		out_file_ << ukf.x_(4) << "\t";	//yaw_rate -est

		//output the measurements
		if (measurement_pack_list[k].sensor_type_
				== MeasurementPackage::LASER) {
			//output the estimation
			out_file_ << measurement_pack_list[k].raw_measurements_(0) << "\t";	//p1 - meas
			out_file_ << measurement_pack_list[k].raw_measurements_(1) << "\t"; //p2 - meas
		} else if (measurement_pack_list[k].sensor_type_
				== MeasurementPackage::RADAR) {
			//output the estimation in the cartesian coordinates
			float ro = measurement_pack_list[k].raw_measurements_(0);
			float phi = measurement_pack_list[k].raw_measurements_(1);
			out_file_ << ro * cos(phi) << "\t";	//p1_meas
			out_file_ << ro * sin(phi) << "\t";	//p2_meas
		}

		//output the gt packages
		out_file_ << gt_pack_list[k].gt_values_(0) << "\t"; //p1 - GT
		out_file_ << gt_pack_list[k].gt_values_(1) << "\t";	//p2 - GT
		out_file_ << gt_pack_list[k].gt_values_(2) << "\t";	//v_abs - GT
		out_file_ << gt_pack_list[k].gt_values_(3) << "\t";	//yaw - GT
		out_file_ << gt_pack_list[k].gt_values_(4) << "\t";	//yaw_dot - GT
		out_file_ << gt_pack_list[k].gt_values_(5) << "\t";	//v1 - GT
		out_file_ << gt_pack_list[k].gt_values_(6) << "\n";	//v2 - GT

In [104]:
import plotly.offline as py
from plotly.graph_objs import *
import pandas as pd
import math
py.init_notebook_mode()

#we aim to visualize the following params: 
#'p1', 'p1est','p1meas', 'p2', 'p2est','p2meas', 'v', 'vest', 'yaw', 'yawest', 'yawrate', 'yawrateest', 'acc', 'yawacc'

#input file fields as they are saved into the UKF output file
my_cols=['p1est','p2est','vest','yawest','yawrateest','p1meas','p2meas','p1','p2','v','yaw', 'yawrate','v1_gt','v2_gt', 'NIS_laser', 'NIS_radar']
with open('../unscented_kalman_filter/data/obj_pose-laser-radar-ukf-output.txt') as f:
    table_ekf_output = pd.read_table(f, sep='\t', header=None, names=my_cols, lineterminator='\n')
    


# Cool

In [105]:
#check the parsed file
table_ekf_output[0:5]

p1est     p2est     vest    yawest  yawrateest    p1meas    p2meas  \
0  0.463227  0.607415  0.10000  0.100000    0.010000  0.463227  0.607415   
1  0.611405  0.532532  1.37992  0.274880    0.020167  0.732611  0.520449   
2  0.877526  0.468509  1.39741  0.203513    0.016616  0.968521  0.405450   
3  0.866279  0.595571  1.47580  0.491820    0.050091  0.746070  0.522039   
4  0.934119  0.629626  1.47926  0.492904    0.049804  0.947752  0.636824   

         p1        p2        v       yaw   yawrate    v1_gt     v2_gt  \
0  0.600000  0.600000  2.19994  0.000000  0.006911  2.19994  0.000000   
1  0.709997  0.600019  2.19975  0.000346  0.013822  2.19975  0.000760   
2  0.819984  0.600095  2.19943  0.001037  0.020730  2.19943  0.002280   
3  0.929956  0.600266  2.19899  0.002073  0.027634  2.19899  0.004559   
4  1.039900  0.600570  2.19842  0.003455  0.034535  2.19841  0.007595   

   NIS_laser  NIS_radar  
0    0.00000    0.00000  
1    0.00000   16.26100  
2    1.61171   16.26100  
3    1.61171    1.78837  
4    0.01784    1.78837

# Visualize and compare the estimated/measurement/groun truth poses in 2D

In [106]:
import plotly.offline as py
from plotly.graph_objs import *




#Ground Truth
trace1 = Scatter(
    x=table_ekf_output['p1'],
    y=table_ekf_output['p2'],
    xaxis='x2',
    yaxis='y2',
    name = 'ground truth position',
    mode = 'markers'      
)


#estimations
trace2 = Scatter(
    x=table_ekf_output['p1est'],
    y=table_ekf_output['p2est'],
    xaxis='x2',
    yaxis='y2',
    name='UKF position estimation',
    mode = 'markers'       
)

#Measurements
trace3 = Scatter(
    x=table_ekf_output['p1meas'],
    y=table_ekf_output['p2meas'],
    xaxis='x2',
    yaxis='y2',
    name = 'position measurements',
    #mode = 'markers'
)


data = [trace1, trace2, trace3]

layout = Layout(
    xaxis2=dict(
   
        anchor='x2',
        title='px in m'
    ),
    yaxis2=dict(
    
        anchor='y2',
        title='py in m'
    )
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename= 'EKF')

# Now let's swithc to something serious 

In [107]:
import plotly.offline as py
from plotly.graph_objs import *

t = range(0,len(table_ekf_output))
## we want: 'p1', 'p1est','p1meas', 'p2', 'p2est','p2meas', 'v', 'vest', 'yaw', 'yawest', 'yawrate', 'yawrateest', 'acc', 'yawacc'
#['p1est','p2est','vest','yawest','yawrateest','p1meas','p2meas','p1','p2','v','yaw', 'yawrate','v1_gt','v2_gt']


trace1 = Scatter(
    x=t,
    y=table_ekf_output['p1'],
    xaxis='x2',
    yaxis='y2',
    name = 'px ground truth in m',
    #mode = 'markers'
)

trace2= Scatter(
    x=t,
    y=table_ekf_output['p1est'],
    xaxis='x2',
    yaxis='y2',
    name = 'px estimated in m ',
    #mode = 'markers'
)

trace3 = Scatter(
    x=t,
    y=table_ekf_output['p1meas'],
    xaxis='x2',
    yaxis='y2',
    name = 'px measured in m ',
    #mode = 'markers'

)



data = [trace1, trace2, trace3]

layout = Layout(
    xaxis2=dict(
   
        anchor='x2',
        title='t'
    ),
    yaxis2=dict(
    
        anchor='y2',
        title='m'
    )
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename= 'EKF')

In [108]:
import plotly.offline as py
from plotly.graph_objs import *

t = range(0,len(table_ekf_output))
## we want: 'p1', 'p1est','p1meas', 'p2', 'p2est','p2meas', 'v', 'vest', 'yaw', 'yawest', 'yawrate', 'yawrateest', 'acc', 'yawacc'
#['p1est','p2est','vest','yawest','yawrateest','p1meas','p2meas','p1','p2','v','yaw', 'yawrate','v1_gt','v2_gt']


trace7 = Scatter(
    x=t,
    y=table_ekf_output['v'],
    xaxis='x2',
    yaxis='y2',
    name = 'ground truth velocity ',
    #mode = 'markers'
)

trace8 = Scatter(
    x=t,
    y=table_ekf_output['vest'],
    xaxis='x2',
    yaxis='y2',
    name = 'estimated velocity ',
    #mode = 'markers'
)



data = [trace7, trace8]

layout = Layout(
    xaxis2=dict(
   
        anchor='x2',
        title='t'
    ),
    yaxis2=dict(
    
        anchor='y2',
        title='m/s'
    )
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename= 'EKF')

In [89]:
import plotly.offline as py
from plotly.graph_objs import *

t = range(0,len(table_ekf_output))
## we want: 'p1', 'p1est','p1meas', 'p2', 'p2est','p2meas', 'v', 'vest', 'yaw', 'yawest', 'yawrate', 'yawrateest', 'acc', 'yawacc'
#['p1est','p2est','vest','yawest','yawrateest','p1meas','p2meas','p1','p2','v','yaw', 'yawrate','v1_gt','v2_gt']


trace7 = Scatter(
    x=t,
    y=table_ekf_output['yaw'],
    xaxis='x2',
    yaxis='y2',
    name = 'ground truth yaw angle ',
    #mode = 'markers'
)

trace8 = Scatter(
    x=t,
    y=table_ekf_output['yawest'],
    xaxis='x2',
    yaxis='y2',
    name = 'estimated yaw angle ',
    #mode = 'markers'
)



data = [trace7, trace8]

layout = Layout(
    xaxis2=dict(
   
        anchor='x2',
        title='t'
    ),
    yaxis2=dict(
    
        anchor='y2',
        title='rad'
    )
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename= 'EKF')

In [88]:
import plotly.offline as py
from plotly.graph_objs import *

t = range(0,len(table_ekf_output))
## we want: 'p1', 'p1est','p1meas', 'p2', 'p2est','p2meas', 'v', 'vest', 'yaw', 'yawest', 'yawrate', 'yawrateest', 'acc', 'yawacc'
#['p1est','p2est','vest','yawest','yawrateest','p1meas','p2meas','p1','p2','v','yaw', 'yawrate','v1_gt','v2_gt']


trace7 = Scatter(
    x=t,
    y=table_ekf_output['yawrate'],
    xaxis='x2',
    yaxis='y2',
    name = 'ground truth yaw rate ',
    #mode = 'markers'
)

trace8 = Scatter(
    x=t,
    y=table_ekf_output['yawrateest'],
    xaxis='x2',
    yaxis='y2',
    name = 'estimated yaw rate',
    #mode = 'markers'
)



data = [trace7, trace8]

layout = Layout(
    xaxis2=dict(
   
        anchor='x2',
        title='t'
    ),
    yaxis2=dict(
    
        anchor='y2',
        title='rad/s'
    )
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename= 'EKF')

In [94]:
import plotly.offline as py
from plotly.graph_objs import *

t = range(0,len(table_ekf_output))
## we want: 'p1', 'p1est','p1meas', 'p2', 'p2est','p2meas', 'v', 'vest', 'yaw', 'yawest', 'yawrate', 'yawrateest', 'acc', 'yawacc'
#['p1est','p2est','vest','yawest','yawrateest','p1meas','p2meas','p1','p2','v','yaw', 'yawrate','v1_gt','v2_gt']


trace2 = Scatter(
    x=t,
    y=table_ekf_output['NIS_laser'],
    xaxis='x2',
    yaxis='y2',
    name = 'NIS laser',
    #mode = 'markers'
)


trace1= Scatter(
    x=[t[0], t[-1]],
    y=[5.991, 5.991],
    xaxis='x2',
    yaxis='y2',
    name = '95 %',
    #mode = 'markers'
)



data = [trace1, trace2]

layout = Layout(
    xaxis2=dict(
   
        anchor='x2',
        title='k'
    ),
    yaxis2=dict(
    
        anchor='y2',
        #title='py'
    )
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename= 'EKF')

In [93]:
import plotly.offline as py
from plotly.graph_objs import *

t = range(0,len(table_ekf_output))
## we want: 'p1', 'p1est','p1meas', 'p2', 'p2est','p2meas', 'v', 'vest', 'yaw', 'yawest', 'yawrate', 'yawrateest', 'acc', 'yawacc'
#['p1est','p2est','vest','yawest','yawrateest','p1meas','p2meas','p1','p2','v','yaw', 'yawrate','v1_gt','v2_gt']


trace2 = Scatter(
    x=t,
    y=table_ekf_output['NIS_radar'],
    xaxis='x2',
    yaxis='y2',
    name = 'NIS radar',
    #mode = 'markers'
)


trace1= Scatter(
    x=[t[0], t[-1]],
    y=[7.815, 7.815],
    xaxis='x2',
    yaxis='y2',
    name = '95 %',
    #mode = 'markers'
)



data = [trace1, trace2]

layout = Layout(
    xaxis2=dict(
   
        anchor='x2',
        title='k'
    ),
    yaxis2=dict(
    
        anchor='y2',
        #title='py'
    )
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename= 'EKF')

In [83]:
import plotly.offline as py
from plotly.graph_objs import *

t = range(0,len(table_ekf_output))
## we want: 'p1', 'p1est','p1meas', 'p2', 'p2est','p2meas', 'v', 'vest', 'yaw', 'yawest', 'yawrate', 'yawrateest', 'acc', 'yawacc'
#['p1est','p2est','vest','yawest','yawrateest','p1meas','p2meas','p1','p2','v','yaw', 'yawrate','v1_gt','v2_gt']


trace1 = Scatter(
    x=t,
    y=table_ekf_output['p1'],
    xaxis='x2',
    yaxis='y2',
    name = 'p1',
    #mode = 'markers'
)

trace2= Scatter(
    x=t,
    y=table_ekf_output['p1est'],
    xaxis='x2',
    yaxis='y2',
    name = 'p1est',
    #mode = 'markers'
)

trace3 = Scatter(
    x=t,
    y=table_ekf_output['p1meas'],
    xaxis='x2',
    yaxis='y2',
    name = 'p1meas',
    #mode = 'markers'
)

trace4= Scatter(
    x=t,
    y=table_ekf_output['p2'],
    xaxis='x2',
    yaxis='y2',
    name = 'p2',
    #mode = 'markers'
)

trace5 = Scatter(
    x=t,
    y=table_ekf_output['p2est'],
    xaxis='x2',
    yaxis='y2',
    name = 'p2est',
    #mode = 'markers'
)

trace6= Scatter(
    x=t,
    y=table_ekf_output['p2meas'],
    xaxis='x2',
    yaxis='y2',
    name = 'p2meas',
    #mode = 'markers'
)

trace7 = Scatter(
    x=t,
    y=table_ekf_output['v'],
    xaxis='x2',
    yaxis='y2',
    name = 'v',
    #mode = 'markers'
)

trace8 = Scatter(
    x=t,
    y=table_ekf_output['vest'],
    xaxis='x2',
    yaxis='y2',
    name = 'vest',
    #mode = 'markers'
)

trace9= Scatter(
    x=t,
    y=table_ekf_output['yaw'],
    xaxis='x2',
    yaxis='y2',
    name = 'yaw',
    #mode = 'markers'
)

trace10 = Scatter(
    x=t,
    y=table_ekf_output['yawest'],
    xaxis='x2',
    yaxis='y2',
    name = 'yawest',
    #mode = 'markers'
)

trace11= Scatter(
    x=t,
    y=table_ekf_output['yawrate'],
    xaxis='x2',
    yaxis='y2',
    name = 'yawrate',
    #mode = 'markers'
)

trace12= Scatter(
    x=t,
    y=table_ekf_output['yawrateest'],
    xaxis='x2',
    yaxis='y2',
    name = 'yawrateest',
    #mode = 'markers'
)

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9, trace10, trace11, trace12]

layout = Layout(
    xaxis2=dict(
   
        anchor='x2',
        title='t'
    ),
    yaxis2=dict(
    
        anchor='y2',
        #title='py'
    )
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename= 'EKF')